In [103]:
import multiprocessing
import ollama

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional

df = pd.read_csv('../dataset/data.csv')

df.head()

,OBSRVTN_NB,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX
0,330560,3/15/2023 11:01,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NaN
1,164330,7/9/2019 10:00,Vehicle Operating Condition,Other - Vehicle Operating Condition,[NAME] trucks with cut out bumbers need a hitc...,NaN
2,265239,8/1/2022 10:15,Suspended Load/Overhead Work,Other - Suspended Load/Overhead Work,Employee rigged a concrete culvert to be off l...,Coached employee that once he had the culverts...
3,404438,10/18/2023 12:06,PPE - Workforce,Other - PPE Workforce,[NAME] tender was not wearing his Hardhat. I r...,NaN
4,83172,3/3/2021 11:00,Climbing - Procedures,"Was a drop zone established, and clearly marked?",A drop zone was not clearly marked by the crew...,NaN


In [104]:
df.set_index('OBSRVTN_NB', inplace=True)
df.fillna("NA", inplace=True)
df['DATETIME_DTM'] = pd.to_datetime(df['DATETIME_DTM'])
df_subsample = df[::2000]
df.head()

,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX
OBSRVTN_NB,,,,,
330560,2023-03-15 11:01:00,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NA
164330,2019-07-09 10:00:00,Vehicle Operating Condition,Other - Vehicle Operating Condition,[NAME] trucks with cut out bumbers need a hitc...,NA
265239,2022-08-01 10:15:00,Suspended Load/Overhead Work,Other - Suspended Load/Overhead Work,Employee rigged a concrete culvert to be off l...,Coached employee that once he had the culverts...
404438,2023-10-18 12:06:00,PPE - Workforce,Other - PPE Workforce,[NAME] tender was not wearing his Hardhat. I r...,NA
83172,2021-03-03 11:00:00,Climbing - Procedures,"Was a drop zone established, and clearly marked?",A drop zone was not clearly marked by the crew...,NA


In [105]:
SYSTEM_PROMPT_FORMAT = """
You are a superintelligent Safety Classification and Learning (SCL) Model 
with the goal classifying field notes written on site by AEP electrical engineers and other essential personnel.

Your task is to reason with the field notes and output a classification for that field note data.

For your context, for the rest of this prompt, when we refer to High Energy Situations, High Energy Incidents, Serious Injury Sustained, and Direct Control present are as follows:
Definition of High Energy Situation:
High energy refers to situations where the physical forces or hazardous conditions involved are strong enough to greatly increase the likelihood of a serious injury or fatality (SIF). These situations can involve various forms of energy, including gravity, motion, mechanical, electrical, pressure, sound, radiation, biological, chemical, or temperature.
The presence of high energy indicates that the conditions pose a significant risk to individuals, typically involving forces or exposures that exceed safe thresholds. For example, situations such as falls from elevation, suspended loads, heavy rotating equipment, explosions, electrical contact exceeding 50 volts, high temperatures above 150°F, or high pressure in excavations over 5 feet deep are typical indicators of high energy, often resulting in severe consequences.
Even when exact measurements like voltage or force are not mentioned, hazardous conditions should be evaluated to determine if the risk level suggests the presence of high energy. In such cases, a reasonable inference can be made based on the description of the incident and the potential for serious harm.
Definition of High Energy Incident:
A high energy incident occurs when a high energy situation results in an actual event, such as an accident, injury, or near-miss. This involves a release of hazardous energy—where the energy changes state or is no longer contained—and the worker comes into contact with or is in proximity to the energy source. Contact means the high energy is transmitted to the body, while proximity refers to the worker being within 6 feet of the hazard with unrestricted egress, or any distance where escape is limited.
Definition of a Serious Injury Sustained:
Serious Injury Sustained includes work related fatalities and life-threatening and life-altering injuries.
Definition of Direct Control Present:
Direct controls are: Targeted (the barrier must specifically target a high-energy source), effective: (it must effectively mitigate exposure to the high-energy source when installed, verified, and used properly (i.e., a SIF reasonable should not occur if these conditions are met)) and have human error: (it must be effective even in the presence of unintentional human error during work that is unrelated to the installation of the control)
Examples of Direct Controls: [lockout/tagout (LOTO), machine guarding, hard physical barriers, fall protection, cover-ups]
Examples that are NOT Direct Controls: [training, warning signs, rules, experience, standard non-specialized personal protective equipment (e.g., hard hats, gloves, boots)]

The field notes have the following schema:
PNT_NM: Point name, the safety criteria being assessed. Forms a primary key when combined with OBSRVN_NB (observation number)
QUALIFIER_TXT: Qualifier text, list of predetermined observations chosen by the reviewer based on the point being assessed
PNT_ATRISKNOTES_TX: Point at-risk notes text, comments left by observer regarding unsafe conditions they found
PNT_ATRISKFOLWUPNTS_TX: Point at-risk follow up notes text, recommended remediation for at-risk conditions observed. This field may be empty (Indicated by "NA"). 

You must classify this data using these classes. The format of the classes is "class": -> "description of class":
{classes}

NOTE: your output should ONLY be the class code and NOTHING else:
"""

USER_PROMPT_FORMAT = """
Below is the field notes:

{field_notes}

REMEMBER:
NOTE: your output should ONLY be the class code and NOTHING else:
"""

In [106]:
class GenerateWeakLabels:
    def __init__(
            self,
            classes: Dict[int, str],
            column_label: str,
            shot_examples: Optional[Dict[str, str]] = None,
            df: pd.DataFrame = df_subsample,
            model: str = 'llama3:latest',
        ):  
        """Generate one decision for an llm to make. """
        predictions = []

        for _, row in df.iterrows():
            prompt_field_notes = "\n".join(f"{key}: {row[key]}" for key in ['PNT_ATRISKNOTES_TX'])#['PNT_NM', 'QUALIFIER_TXT', 'PNT_ATRISKNOTES_TX', 'PNT_ATRISKFOLWUPNTS_TX'])
            #print(prompt_field_notes)

            output = ollama.chat(
                model    = model, 
                messages = self._build_shot_prompts(classes, prompt_field_notes, shot_examples),
                options  = {'temperature': 0, 'num_predict': 1}
            )

            #print(output['message']['content'])
            predictions.append(output['message']['content'])

        df[column_label] = predictions

    def _schedule_classification_queries(self):
        ...
        
    def _build_shot_prompts(self, classes, prompt_field_notes, shot_examples):
        prompt_classes = "\n".join(f"{k}: -> {v}" for k, v in classes.items())

        messages = [{'role': 'system', 'content': SYSTEM_PROMPT_FORMAT.format(classes=prompt_classes)}]

        if shot_examples is None:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes,)})
            #self._pretty_print_messages(messages)
            return messages

        for example in shot_examples:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=example)})
            messages.append({'role': 'assistant', 'content': str(shot_examples[example])})

        messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes)})
        #self._pretty_print_messages(messages)
        return messages
    
    def _pretty_print_messages(self, messages):
        for message in messages:
            role = message['role'].capitalize()
            content = message['content']
            print(f"{role}: {content}\n")

In [107]:
classes = {
    0: 'Def: High Energy Serious Injury Sustained (HSIF). This means that there was a High Energy Incident AND there was a Serious Injury Sustained.',
    1: 'Def: Low Energy Serious Injury Sustained (LSIF). This means that there was NOT a High Energy Present AND there was a Serious Injury Sustained.',
    2: 'Def: Potential Serious Injury Sustained (PSIF). This means that there was a High Energy Incident AND there was NOT a Serious Injury Sustained.',
    3: 'Def: None of the above criteria defs (HSIF, LSIF, PSIF) are met.',
}

multishot = {
    """
    PNT_NM: Climbing - Procedures
    QUALIFIER_TXT: Was a drop zone established, and clearly marked?
    PNT_ATRISKNOTES_TX: Workers were hoisting beams and steel onto a scaffold. A certified mechanic operated an air hoist to lift the beam. As the lift was performed, the rigging was caught under an adjacent beam. Under the increasing tension, the cable snapped and struck a second employee in the leg, fully fracturing his femur. An investigation indicated that the rigging was not properly inspected before the lift.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "0",
    """
    PNT_NM: Housekeeping - Generation
    QUALIFIER_TXT: Job site hazards, Tripping Hazards
    PNT_ATRISKNOTES_TX: An employee was descending a staircase and when stepping down from the last step she rolled her
    ankle on an extension cord on the floor. She suffered a torn ligament and a broken ankle that resulted in
    persistent pain for more than a year.
    PNT_ATRISKFOLWUPNTS_TX: NA 
    """: "1",
    """
    PNT_NM: Workplace Conditions Addressed
    QUALIFIER_TXT: Voltage being worked discussed
    PNT_ATRISKNOTES_TX: A crew was working near a sedimentation pond on a rainy day. The boom of the trac-hoe was within 3
    feet of a live 12kV line running across the road. No contact was made because a worker intervened and
    communicated with the operator.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "3",
    """
    PNT_NM: Did you recognize additional slip, trip, or fall hazards that had not already been recognized and mitigated? If so, please select or describe these hazards in the At-Risk notes.
    QUALIFIER_TXT: Awareness of environment
    PNT_ATRISKNOTES_TX: [NAME] was working a near by cliff that had about a 20' drop off, crew didn't discuss as a hazard on briefing, i discussed with GF and he told the foreman to make the corrections and place something out there to give crews a visual.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "2",
}

GenerateWeakLabels(classes=classes, shot_examples=multishot, column_label='cls')

df_subsample

/var/folders/1h/mw4kschx1jjdmd14bjl9nv4r0000gn/T/ipykernel_51180/1591017843.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_label] = predictions


,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX,cls
OBSRVTN_NB,,,,,,
330560,2023-03-15 11:01:00,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NA,0
414942,2023-11-06 10:51:00,PPE - Workforce,AEP Required (Standard),[NAME] employee had prescription non safety gl...,NA,3
253483,2022-06-28 13:30:00,Housekeeping in order,Job site hazards,[NAME] further discussion regarding the uneven...,NA,3
48181,2020-08-12 14:16:00,Correct work area traffic control,Other - Work Area Protection,Some of the signs labeling has pealed off and ...,NA,3
446761,2024-03-15 17:24:00,Personal Safety,Line of fire,Reminded crew of avoiding the road side of the...,NA,3
209710,2022-01-18 15:45:00,PPE,"Other - PPE, Safety glasses adequate",Employee operating Digger [NAME] not wearing s...,NA,3
476168,2024-07-02 13:30:00,"15) If new hazards were identified, or if cond...",[Notes Required for At-Risk Conditions],job briefing made note that ribbons were used ...,NA,3
100026,2021-04-22 16:30:00,Qualified observer,Observer engagement,"I SPOKE WITH JASON, WHO WAS THE QO, ABOUT MAKI...",NA,3
34428,2020-05-12 12:30:00,Appropriate work area traffic control (Forestry),"Appropriate signs out, Cones compliant",[NAME] was parked on side of the road without ...,NA,3


In [108]:
test_data = pd.DataFrame({
    'PNT_ATRISKNOTES_TX': [
        "An employee was on the top of a de-energized transformer at 25 feet of height with a proper fall arrest system. While working, she tripped on a lifting lug, falling within 2 feet from an unguarded edge. When the employee landed, she sprained her wrist.",
        "An employee contracted West Nile Virus after being bitten by a mosquito while at work in a boggy area. Because of the exposure, the employee was unconscious and paralyzed for a two-week period.",
        "An employee was working alone and placed an extension ladder against the wall. When he reached 10 feet of height, the ladder feet slid out and he fell with the ladder to the floor. The employee was taken to the hospital for a bruise to his right leg and remained off duty for three days.",
        "A crew was closing a 7-ton door on a coal crusher. As the door was lowered, an observer noticed that the jack was not positioned correctly and could tip. The observer also noted that workers were nearby, within 4 feet of the jack.",
        "Workers were hoisting beams and steel onto a scaffold. A certified mechanic operated an air hoist to lift the beam. As the lift was performed, the rigging was caught under an adjacent beam. Under the increasing tension, the cable snapped and struck a second employee in the leg, fully fracturing his femur. An investigation indicated that the rigging was not properly inspected before the lift.",
        "A dozer was operating on a pet coke pile and slid down an embankment onto the cab after encountering a void in the pile. The operator was wearing his seat belt, and the roll cage kept the cab from crushing. No workers or machinery were nearby, and no injuries were sustained.",
        "A master electrician was called to work on a new 480-volt service line in a commercial building. When working on the meter cabinet, the master electrician had to position himself awkwardly between the cabinet and a standpipe. He was not wearing an arc-rated face shield, balaclava, or proper gloves. During the work, an arc flash occurred, causing third-degree burns to his face.",
        "An employee was descending a staircase and when stepping down from the last step she rolled her ankle on an extension cord on the floor. She suffered a torn ligament and a broken ankle that resulted in persistent pain for more than a year.",
        "A crew was working near a sedimentation pond on a rainy day. The boom of the trac-hoe was within 3 feet of a live 12kV line running across the road. No contact was made because a worker intervened and communicated with the operator.",
        "A crew was working in a busy street to repair a cable fault. During the work, the journeyman took a step back from the truck outside of the protected work zone into oncoming traffic. A driver slammed on his brakes and stopped within one foot of the journeyman. No injuries were sustained."
    ],
    'PNT_ATRISKFOLWUPNTS_TX': ["NA"] * 10  # No follow-up notes for all test cases
})

GenerateWeakLabels(classes=classes, shot_examples=multishot, df=test_data, column_label='cls')
test_data

,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX,cls
0,An employee was on the top of a de-energized t...,NA,0
1,An employee contracted West Nile Virus after b...,NA,0
2,An employee was working alone and placed an ex...,NA,0
3,A crew was closing a 7-ton door on a coal crus...,NA,0
4,Workers were hoisting beams and steel onto a s...,NA,0
5,A dozer was operating on a pet coke pile and s...,NA,0
6,A master electrician was called to work on a n...,NA,0
7,An employee was descending a staircase and whe...,NA,1
8,A crew was working near a sedimentation pond o...,NA,0
9,A crew was working in a busy street to repair ...,NA,0
